# Carousel mechanism design
Project: *LuSEE-Night* \
System: *Antenna Carousel* \
Author: *Joe Silber* \
Institution: *Lawrence Berkeley National Lab (LBNL)* \
Date: *July 2022*

Abstract: Assessment of carousel (a.k.a. "turntable") mechanism and torques for the LuSEE-Night lunar instrument. The radio antennas rotate on a platform. Key design parameters and options for the rotation mechanism are considered here.

## Preliminaries

### Units and coordinate systems
Except where otherwise specified...
- all units are in meters, kilograms, newtons, and seconds
- all inertias and positions are with respect to a global coordinate system, with Z axis concentric to rotation axis

### Python initializations

In [1]:
import math

### General material properties

In [2]:
rho_ss = 8000  # stainless steel density
rho_al = 2700  # aluminm density

### Carbon fiber properties

In [3]:
rho_f = 2170  # carbon fiber density, e.g. CN-80
rho_m = 1170  # resin matrix density, e.g. EX-1515
Mm = 0.32  # matrix mass fraction, i.e. prepreg resin content from vendor
Vv = 0.005  # as-cured void fraction
Vf = (1 - Vv) / (1 + rho_f / rho_m * Mm / (1 - Mm))  # fiber volume fraction, assuming no bleed (conservative)
cfrp_density = rho_f * Vf + rho_m * (1 - Vf - Vv)
print(f'Estimated carbon fiber reinforced plastic (CFRP) density = {cfrp_density:.1f} kg/m^3, volume fraction = {Vf*100:.1f}%')

Estimated carbon fiber reinforced plastic (CFRP) density = 1695.4 kg/m^3, volume fraction = 53.1%


## Key mechanical design criteria
The design criteria below will be used in sizing components and ensuring sufficient mechanical factors of safey. These criteria are informed by key recommendations in NASA's [General Environmental Verification Standards (GEVS)](https://standards.nasa.gov/sites/default/files/standards/GSFC/B/0/gsfc-std-7000b_signature_cycle_04_28_2021_fixed_links.pdf) (accessed 2022-07-09).

### Structural load limit
The limit loads for the carousel mechanism are derived from GEVS Table 2.4-3,
*Generalized Random Vibration Test Levels*. For components weighing less than 22.7 kg, an overall qualification level (in $G_{rms}$) is directly specified. We then apply a proof factor per section 2.4.1.4.1, and a design factor to get the design limit. The design factor accommodated uncertainty in material properties and tolerances in the proof test method. These limits are applied for nominal accelerations in 3 orthogonal axes.

In [14]:
Grms_qual = 14.1
proof_factor = 1.25
design_factor = 1.2
Grms_design = Grms_qual * proof_factor * design_factor
print(f'Design load level = {Grms_design:.1f} G(rms)')

Design load level = 21.1 G(rms)


### Torque margin
Torque margin ($TM$) for the carousel turntable is assessed per GEVS section 2.4.5.3. Two safety factors, $FS_k$ and $FS_v$ are applied to the "known" and "variable" torques in the system. The available driving torque $T_{avail}$ at the motor (not including gearhead) is compared according to:

$$TM = \frac{T_{avail}}{FS_k \sum T_{known} + FS_v \sum T_{variable}} - 1$$

These torques are characterized in GEVS as:

> **Driving Torques**
>
> $T_{avail}$ = Minimum Available Torque or Force generated by the mechanism at worst case environmental conditions at any time in its life. If motors are used in the system, $T_{avail}$ shall be determined at the output of the motor, not including gear heads or gear trains at its output based on minimum supplied motor voltage. $T_{avail}$ similarly applies to other actuators such as springs, pyrotechnics, solenoids, heat actuated devices, etc.
> 
> **Resistive Torques**
>
> $\sum T_{known}$ = Sum of the fixed torques or forces that are known and quantifiable such as
accelerated inertias ($T = I \alpha$) and not influenced by friction, temperature, life, etc. A constant Safety Factor is applied to the calculated torque.
>
> $\sum T_{variable}$ = Sum of the torques or forces that may vary over environmental conditions and life such as static or dynamic friction, alignment effects, latching forces, wire harness
loads, damper drag, variations in lubricant effectiveness, including degradation or depletion of lubricant over life, etc.

Here we use the highest (PDR phase) torque safety factor values from GEVS:

In [5]:
FSk = 2.0  # preliminary design review phase
           # drops to 1.5 at critical design review
           # remains 1.5 at acceptance / qualification testing

FSv = 4.0  # preliminary design review phase
           # drops to 3.0 at critical design review
           # drops to 2.0 at acceptance / qualification testing

## Payload
Total mass budget for the carousel system is 10 kg. Some fraction of this mass (in particular the drive motor and transmission) will be static.

As of this writing (July 2022) we have not yet lightweighted many components; the total payload of the items listed below may somewhat exceed that final budget. For the purposes of sizing the drive system components, this is acceptable.

For the purposes of this document, we subdivide the rotating payload into the following estimated masses, moments, and transmission ratios:

### Turntable plate

In [6]:
plate_od = 0.500  # outer diameter
plate_id = 0.079  # inner diameter
plate_t = 0.010  # thickness
plate_rho = cfrp_density  # density
plate_r2, plate_r1 = plate_od/2, plate_id/2  # radii
plate_mass = plate_rho * plate_t * math.pi * (plate_r2**2 - plate_r1**2)
plate_inertia = 0.5 * plate_mass * (plate_r1**2 + plate_r2**2)
print(f'Turntable: mass = {plate_mass:.3f} kg, inertia = {plate_inertia:.4} kg/m^2')

Turntable: mass = 3.246 kg, inertia = 0.104 kg/m^2


### Antenna deployer and pre-amp
Mass and inertia estimates are based on the CAD preliminary design model (July 2022).

#### Stowed configuration:
![Stowed mass and inertia](inertia-deployer-and-preamp-stowed.png)

#### Deployed configuration (3 m extension):
![Deployed mass and inertia](inertia-deployer-and-preamp-deployed3m.png)

Note that the masses shown here are not quite identical, due to internal details of the preliminary model. For the conservative purposes of this study, a rough adjustment is made where the larger mass value (stowed) is used to linearly scale up the deployed mass and inertia.

In [7]:
stowed_mass = 3.148
stowed_inertia = 0.1272
deployed_mass_as_modeled = 2.924
deployed_inertia_as_modeled = 4.203
deployed_adjustment = stowed_mass / deployed_mass_as_modeled
deployed_mass = deployed_mass_as_modeled * deployed_adjustment
deployed_inertia = deployed_inertia_as_modeled * deployed_adjustment
print(f'Deployed single antenna + pre-amp (adjusted): mass = {deployed_mass:.3f} kg, inertia = {deployed_inertia:.4} kg/m^2')

Deployed single antenna + pre-amp (adjusted): mass = 3.148 kg, inertia = 4.525 kg/m^2


### Central shaft and bearings
These rotating components are lumped together and approximated as a single tube of metal.

In [8]:
central_od = 0.080  # outer diameter
central_id = 0.070  # inner diameter
central_length = 0.020  # length
central_r1, central_r2 = central_id/2, central_od/2  # radii
central_rho = rho_ss
central_mass = central_rho * central_length * math.pi * (central_r2**2 - central_r1**2)
central_inertia = 0.5 * central_mass * (central_r1**2 + central_r2**2)
print(f'Central shaft and bearing: mass = {central_mass:.3f} kg, inertia = {central_inertia:.4} kg/m^2')

Central shaft and bearing: mass = 0.188 kg, inertia = 0.0002662 kg/m^2


### Parallel-axis transmission

### Motor

## Friction

### Dust seals

Some dust seals will be required to prevent lunar dust from accessing the bearing and transmission. These impose some friction load on the assembly.

Here I assume the seal contact is made against a flat face by annular, thin-walled polyimide. In this approach, there would be one seal near the central bearing, and another further out in diameter. These would bear either on the rotating turntable be mounted to the turntable and bear on its opposing counterpart (the top plate or the central shaft support). Preload force would be low, since only dust needs to be excluded.

In [9]:
seal_contact_radii = [0.080, 0.250]  # annuli of contact for the two seals
seal_contact_width = 0.003  # rough guess as to desirable contact width, will depend on seal design details
seal_contact_pressure_millibar = 10  # rough guess as to desirable clamping pressure (per seal)
seal_contact_pressure = seal_contact_pressure_millibar * 100
seal_contact_areas = [2 * math.pi * radius * seal_contact_width for radius in seal_contact_radii]
seal_clamping_forces = [seal_contact_pressure * area for area in seal_contact_areas]
for i in range(len(seal_contact_radii)):
    print(f'Dust seal at radius {seal_contact_radii[i]:.3f} m: clamping force = {seal_clamping_forces[i]:.1f} N')

Dust seal at radius 0.080 m: clamping force = 1.5 N
Dust seal at radius 0.250 m: clamping force = 4.7 N


Datasheet values for the dynamic friction coefficient of [Vespel SP-3](https://www.curbellplastics.com/Research-Solutions/DuPont-Vespel-SP-3) are listed at 0.17 - 0.25 depending on pressure and speed. I presume a fixed, static value of 0.3 for the purposes of this calculation.

In [10]:
seal_mu = 0.3


In [11]:



payloads = {'motor internal':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },
            
            'motor external':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },
            
            'main shaft bearing':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },            
            
            'main shaft':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },    
            
            'plate':
                {'mass': plate_mass,
                 'inertia': 0.0,
                 'count': 1,
                 'ratio': 1.0,
                },
            
            'antenna (deployed)':
                {'mass': 0.0,
                 'inertia': 0.0,
                 'count': 4,
                 'ratio': 1.0,
                },
           }

## Bearings

### Bearing type and size
The preliminary design (July 2022) packaging acommodates a pair of Kaydon KA025AR0 bearings:

![Kaydon KA0xxAR0](kaydon-KA0xxAR0-open-A-bearings.png)

This is a thin section, angular contact, open bearing, with bore 2.5\" (63.5 mm), O.D. 3.0\" (76.2 mm), and thickness 0.25\" (6.35 mm). Mass is low, ~0.12 lbs = 55 g. We utilize the common design pattern of mounting in pairs, to provides tilt resistance.

Stainless steel options (may have custom lead-times and will be more expensive) are available. Replace "K" with "S" in the part number.

Kaydon also offers an attractive "Type X" four-point contact bearing. We have previously used such a bearing (at a much larger scale) in the Dark Energy Spectroscopic Instrument (DESI) ADC rotator mechanism. The type X controls 5 degrees of freedom in a single unit, rather than the typical angular contact duplex. This has the advantage of fewer parts and less weight, as well as removing the need to control thrust preload in the mounting between the two separate bearing units. A type X unit of similar package size as KA025AR0 would be KA025XP0:

![Kaydon KA0xxXP0](kaydon-KA0xxXP0-open-X-bearings.png)

Again, replace "K" with "S" for equivalent stainless option.

### Integrated bearing seals

Various seals are available, which we will utilize, to contain lubricant and exclude lunar dust.

As of July 2022, the precise seal material is to be determined (TBD).

Kaydon's standard seal is nitrile rubber, which we note may need further investigation regarding cryogenic survival temperatures. I would feel most comfortable with polyimide, but need to speak with the vendor.

Seals may be external (between housing and shaft, surrounding the bearing) or integrated in the bearing unit itself. For example, Kaydon offers some models with an integrated double-seal (see table below).

![Kaydon JA0xxXP0](kaydon-JA0xxXP0-sealed-X-bearings.png)

However, the Kaydon line-up does not have sealed bearings in angular contact configurations. 

For sealed bearings, replace "J" with "W" in the part number for the stainless option.

### External bearing seals

If the vendor lacks proven experience with integrated bearing seals in our temperature range (100 K - 400 K), then I prefer that we make our own external seals, from a material with which we have confidence. This way we can move directly to the next phases of testing and integration, as well as have flexibility and visibility as to what is happening.

If we machine our own seals one might consider using [Vespel SP-3](https://www.curbellplastics.com/Research-Solutions/DuPont-Vespel-SP-3). This is a MoS<sub>2</sub>-filled polyimide with a long history of use in space applications, with low creep, high strength, and low coefficient of thermal expansion (for a polymer), and which should survive the temperature extremes.

A concern is that the friction coefficient of SP-3 in vacuum is reported to be very low (0.03) in the Dupont datasheet. We probably in fact desire some consistent friction whether testing in lab air or operating in vacuum. This friction will help keeping the position of the turntable stable when powered off. Also I believe SP-3 may not be available in large enough diameter. Hence it would need to be some polyimide film, presumably.

### Turntable bearings

There are some bearings available with more integrated turntable functionality, for example including an external gear profile. An attractive option may be Kaydon T01-00325EAA:

![Kaydon turntables - basic](kaydon-turntable-bearings-basic.png)

![Kaydon turntables - external gears](kaydon-turntable-bearings-extgear.png)

![Kaydon turntables - illustrations](kaydon-turntable-illustrations.png)

These turntables are not off-the-shelf parts. Cost and lead-time would be TBD. Performance is not particularly any better than the other options; the attraction here would be the compact packaging. The gearing would need some exterior protection from lunar dust.

### Bearings conclusion

*(?) Below, for the purposes of preliminary overall system torque calculations, we will presume selection of a single Type-X, nitrile double-sealed, stainless steel Kaydon bearing, i.e. WA025XP0. (?)*

In [12]:
# Bearing friction
mu_bearing = 0.

## References
- [NASA General Environmental Verification Standards (GEVS)](https://standards.nasa.gov/sites/default/files/standards/GSFC/B/0/gsfc-std-7000b_signature_cycle_04_28_2021_fixed_links.pdf)
- [Kaydon turntable bearings](https://www.kaydonbearings.com/RealiSlim_TT_bearings.htm)
- [Kaydon sealed, slim bearings](https://www.kaydonbearings.com/RealiSlim_sealed_bearings.htm)